In [ ]:
import pandas as pd

sms = pd.read_csv("/kaggle/input/sms-spam-collection-dataset/spam.csv", encoding='latin-1')
sms.head()

In [ ]:
sms =sms.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1)

In [ ]:
sms.rename(columns={'v1': 'Label'}, inplace = True)

In [ ]:
sms.rename(columns={'v2': 'message'}, inplace = True)

In [ ]:
import seaborn as sns

In [ ]:
sns.countplot(sms.Label)

In [ ]:
sms.shape

In [ ]:
sms['message']=sms['message'].str.lower()
sms

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
import string
string.punctuation

In [ ]:
def remove_punct(sms):
    text_tok= word_tokenize(sms)
    l=[]
    for token in text_tok : 
        if not token in string.punctuation : 
            l.append(token)
            
    resultat= " ".join(l)
    return resultat

In [ ]:
sms.message=sms.message.apply(remove_punct)
sms

In [ ]:
from nltk.corpus import stopwords
#Télècharger les stopwords
stop=set(stopwords.words('english'))
stop

In [ ]:
def remove_punct(sms):
    text_tok= word_tokenize(sms)
    l=[]
    for token in text_tok : 
        if not token in string.punctuation : 
            l.append(token)
            
    resultat= " ".join(l)
    return resultat

In [ ]:
sms.message=sms.message.apply(remove_punct)
sms

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [ ]:
def lemm(sms):
    text_tok= word_tokenize(sms)
    l=[]
    for word in text_tok:
            l.append(lemmatizer.lemmatize(word))
    resultat= " ".join(l)
    return resultat

In [ ]:
sms.message=sms.message.apply(lemm)
sms

**Bag of Word**

In [ ]:
corpus=sms['message'].values
corpus

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
bw_vect = CountVectorizer()
# tokenize et construire le vocabulaire
bw_fit=bw_vect.fit(corpus)
# vectoriser les mots
bw_corpus = bw_fit.transform(corpus)

In [ ]:
bw_data=pd.DataFrame(bw_corpus.toarray(),columns=bw_fit.get_feature_names())
bw_data

**TD-IDF**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#Initialiser les paramètres du vectoriseur
tf_vect = TfidfVectorizer(max_features=5000)
#Apprendre le vocabulaire du vectoriseur basé sur le paramètre initialisé
tfidf_fit=tf_vect.fit(corpus)
#Vectoriser le corpus
tfidf_corpus= tfidf_fit.transform(corpus)

In [ ]:
tfidf_data=pd.DataFrame(tfidf_corpus.toarray(),columns=tfidf_fit.get_feature_names())
tfidf_data

**Arbre de décision Bag of Word**

In [ ]:
from sklearn.model_selection import train_test_split
Y= sms.Label
X= bw_data
# Split train / test data :
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [ ]:
from sklearn import tree
tree_model = tree.DecisionTreeClassifier()
tree_model = tree.DecisionTreeClassifier(max_depth = 2)
tree_model=tree_model.fit(X_train, Y_train)

In [ ]:
Y_train.value_counts()

In [ ]:
names = ['ham', 'spam']

In [ ]:
Y_predict=tree_model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
mat = confusion_matrix(Y_predict, Y_test)
print(mat)

In [ ]:
#plt.figure(figsize=(15,5))
sns.heatmap(mat, annot=True,  xticklabels=names, yticklabels=names)
plt.xlabel('Test')
plt.ylabel('Predicted')

In [ ]:
a_CART = accuracy_score(Y_test,Y_predict)
print("L'accuracy score du modèle CART est de : ",a_CART)

**Arbre de décision TD-IDF**

In [ ]:
from sklearn.model_selection import train_test_split
Y= sms.Label
X= tfidf_data
# Split train / test data :
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [ ]:
from sklearn import tree
tree_model = tree.DecisionTreeClassifier()
tree_model = tree.DecisionTreeClassifier(max_depth = 2)
tree_model=tree_model.fit(X_train, Y_train)

In [ ]:
Y_predict=tree_model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
mat = confusion_matrix(Y_predict, Y_test)
print(mat)

In [ ]:
#plt.figure(figsize=(15,5))
sns.heatmap(mat, annot=True,  xticklabels=names, yticklabels=names)
plt.xlabel('Test')
plt.ylabel('Predicted')

In [ ]:
a_CART = accuracy_score(Y_test,Y_predict)
print("L'accuracy score du modèle CART est de : ",a_CART)

**Gridsearch**

In [ ]:
from sklearn.model_selection import GridSearchCV
import numpy as np

In [ ]:
num_leafs = [2,4,6,8,10] # nombre minimum d'observation pour chaque split
depths = np.arange(10,40,5) # The maximum depth of the tree
num_features = np.arange(1,X.shape[1]) # candidates for split 


param_grid = [{'max_depth':depths}]

In [ ]:
grid_tree= GridSearchCV(estimator=tree.DecisionTreeClassifier(),param_grid=param_grid,scoring='accuracy',cv=10)
grid_tree.fit(X_train, Y_train)
best_model_tree = grid_tree.best_estimator_

In [ ]:
grid_tree.best_params_

**Forêt d'arbres décisionnels (Random forest)**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
Rf_model = RandomForestClassifier()
Rf_model=Rf_model.fit(X_train, Y_train)

In [ ]:
Y_predict=Rf_model.predict(X_test)

In [ ]:
mat = confusion_matrix(Y_predict, Y_test)
sns.heatmap(mat, annot=True,  xticklabels=names, yticklabels=names)
plt.xlabel('Test')
plt.ylabel('Predicted')